# Introducing the course project

## The situation

You and your rich Uncle Jimmy just purchased Maven Movies, a brick and mortar DVD Rental
SITUATION business. Uncle Jimmy put up the money, and you’re in charge of the day-to-day operations.

## The Brief

As a new owner, you’ll need to learn everything you can about your business: your product
inventory, your staff, your customer purchase behaviors, etc.
You have access to the entire Maven Movies SQL database, but the remaining employees are
not able to give you much direction. You’ll need to analyze everything on your own.

## The Objective

#### Use MySQL to:

• Access and explore the Maven Movies database

• Develop a firm grasp of the 16 database tables and how they relate to each other

• Analyze all aspects of the company’s data, including transactions, customers, staff, etc.


In [12]:
import pandas as pd
import pymysql

conn = pymysql.connect(host='localhost', user='Lauro', password='Lauro12345', db='mavenmovies')

In [13]:
mycursor = conn.cursor() 
  
mycursor.execute("Show tables;") 
  
myresult = mycursor.fetchall() 
  
for x in myresult: 
    print(x) 

('actor',)
('actor_award',)
('actor_info',)
('address',)
('advisor',)
('category',)
('city',)
('country',)
('customer',)
('customer_list',)
('film',)
('film_actor',)
('film_category',)
('film_list',)
('film_text',)
('inventory',)
('investor',)
('language',)
('nicer_but_slower_film_list',)
('payment',)
('rental',)
('sales_by_film_category',)
('sales_by_store',)
('staff',)
('staff_list',)
('store',)


##### 1.	We will need a list of all staff members, including their first and last names,email addresses, and the store identification number where they work. 


In [14]:
pd.read_sql("""SELECT first_name, 
    last_name, 
    email, 
    store_id
    FROM staff; """, conn)

,first_name,last_name,email,store_id
0,Mike,Hillyer,Mike.Hillyer@sakilastaff.com,1
1,Jon,Stephens,Jon.Stephens@sakilastaff.com,2


##### 2.	We will need separate counts of inventory items held at each of your two stores. 

In [15]:
pd.read_sql("""SELECT store_id, 
               COUNT(inventory_id) AS inventory_items
               FROM inventory
               GROUP BY store_id; """, conn)

,store_id,inventory_items
0,1,2270
1,2,2311


###### 3.	We will need a count of active customers for each of your stores. Separately, please. 

In [16]:
pd.read_sql(""" SELECT store_id, 
                COUNT(customer_id) AS active_customers
                FROM customer
                WHERE active = 1
                GROUP BY store_id;""", conn)

,store_id,active_customers
0,1,318
1,2,266


###### 4.	In order to assess the liability of a data breach, we will need you to provide a count of all customer email addresses stored in the database. 

In [21]:
pd.read_sql("""SELECT COUNT(email) AS emails
               FROM customer; """, conn)

,emails
0,599


##### 5.	We are interested in how diverse your film offering is as a means of understanding how likely  you are to keep customers engaged in the future. Please provide a count of unique film titles  you have in inventory at each store and then provide a count of the unique categories of films you provide. 

In [22]:
pd.read_sql(""" SELECT store_id, 
                COUNT(DISTINCT film_id) AS unique_films
                FROM inventory
                GROUP BY store_id;""", conn)

,store_id,unique_films
0,1,759
1,2,762


In [23]:
pd.read_sql(""" SELECT 
                COUNT(DISTINCT name) AS unique_categories
                FROM category; """, conn)

,unique_categories
0,16


##### 6.	We would like to understand the replacement cost of your films. Please provide the replacement cost for the film that is least expensive to replace, the most expensive to replace, and the average of all films you carry.

In [24]:
pd.read_sql(""" SELECT 
                MIN(replacement_cost) AS least_expensive, 
                MAX(replacement_cost) AS most_expensive, 
                AVG(replacement_cost) AS average_replacement_cost
                FROM film;""", conn)

,least_expensive,most_expensive,average_replacement_cost
0,9.99,29.99,19.984


###### 7. We are interested in having you put payment monitoring systems and maximum payment processing restrictions in place in order to minimize the future risk of fraud by your staff. Please provide the average payment you process, as well as the maximum payment you have processed.

In [25]:
pd.read_sql(""" SELECT
                AVG(amount) AS average_payment, 
                MAX(amount) AS max_payment
                FROM payment;""", conn)

,average_payment,max_payment
0,4.200667,11.99


###### 8.	We would like to better understand what your customer base looks like.  Please provide a list of all customer identification values, with a count of rentals they have made all-time, with your highest volume customers at the top of the list.

In [26]:
pd.read_sql(""" SELECT 
                customer_id, 
                COUNT(rental_id) AS number_of_rentals
                FROM rental
                GROUP BY customer_id
               ORDER BY COUNT(rental_id) DESC; """, conn)

,customer_id,number_of_rentals
0,148,46
1,526,45
2,236,42
3,144,42
4,75,41
...,...,...
594,136,15
595,110,14
596,281,14
597,61,14


###### 9. My partner and I want to come by each of the stores in person and meet the managers. Please send over the managers’ names at each store, with the full address of each property (street address, district, city, and country please).  

In [27]:
pd.read_sql("""SELECT 
               staff.first_name AS manager_first_name, 
               staff.last_name AS manager_last_name,
               address.address, 
               address.district, 
               city.city, 
               country.country
               FROM store
               LEFT JOIN staff ON store.manager_staff_id = staff.staff_id
               LEFT JOIN address ON store.address_id = address.address_id
               LEFT JOIN city ON address.city_id = city.city_id
               LEFT JOIN country ON city.country_id = country.country_id;""",conn)

,manager_first_name,manager_last_name,address,district,city,country
0,Mike,Hillyer,47 MySakila Drive,Alberta,Lethbridge,Canada
1,Jon,Stephens,28 MySQL Boulevard,QLD,Woodridge,Australia


###### 10.	I would like to get a better understanding of all of the inventory that would come along with the business. Please pull together a list of each inventory item you have stocked, including the store_id number, the inventory_id, the name of the film, the film’s rating, its rental rate and replacement cost. 

In [28]:
pd.read_sql(""" SELECT 
                inventory.store_id, 
                inventory.inventory_id, 
                film.title, 
                film.rating, 
                film.rental_rate, 
                film.replacement_cost
                FROM inventory
                LEFT JOIN film
                ON inventory.film_id = film.film_id;""",conn)

,store_id,inventory_id,title,rating,rental_rate,replacement_cost
0,1,1,ACADEMY DINOSAUR,PG,0.99,20.99
1,1,2,ACADEMY DINOSAUR,PG,0.99,20.99
2,1,3,ACADEMY DINOSAUR,PG,0.99,20.99
3,1,4,ACADEMY DINOSAUR,PG,0.99,20.99
4,1,16,AFFAIR PREJUDICE,G,2.99,26.99
...,...,...,...,...,...,...
4576,2,4573,ZOOLANDER FICTION,R,2.99,28.99
4577,2,4578,ZORRO ARK,NC-17,4.99,18.99
4578,2,4579,ZORRO ARK,NC-17,4.99,18.99
4579,2,4580,ZORRO ARK,NC-17,4.99,18.99


##### 11.	From the same list of films you just pulled, please roll that data up and provide a summary level overview of your inventory. We would like to know how many inventory items you have with each rating at each store. 

In [29]:
pd.read_sql(""" SELECT 
                inventory.store_id, 
                film.rating, 
                COUNT(inventory_id) AS inventory_items
                FROM inventory
                LEFT JOIN film
                ON inventory.film_id = film.film_id
                GROUP BY 
                inventory.store_id,
                film.rating;""",conn)

,store_id,rating,inventory_items
0,1,G,394
1,1,PG,444
2,1,PG-13,525
3,1,R,442
4,1,NC-17,465
5,2,G,397
6,2,PG,480
7,2,PG-13,493
8,2,R,462
9,2,NC-17,479


##### 12. Similarly, we want to understand how diversified the inventory is in terms of replacement cost. We want to see how big of a hit it would be if a certain category of film became unpopular at a certain store.We would like to see the number of films, as well as the average replacement cost, and total replacement cost, sliced by store and film category. 

In [30]:
pd.read_sql(""" SELECT 
                store_id, 
                category.name AS category, 
                COUNT(inventory.inventory_id) AS films, 
                AVG(film.replacement_cost) AS avg_replacement_cost, 
                SUM(film.replacement_cost) AS total_replacement_cost
                FROM inventory
                LEFT JOIN film
                ON inventory.film_id = film.film_id
                LEFT JOIN film_category
                ON film.film_id = film_category.film_id
                LEFT JOIN category
                ON category.category_id = film_category.category_id
                GROUP BY 
                store_id, 
                category.name
                ORDER BY 
                SUM(film.replacement_cost) DESC;""",conn)

,store_id,category,films,avg_replacement_cost,total_replacement_cost
0,2,Sports,181,20.697182,3746.19
1,1,Action,169,21.191183,3581.31
2,1,Drama,162,21.934444,3553.38
3,2,Animation,174,19.995747,3479.26
4,2,Documentary,164,20.544878,3369.36
5,1,Sports,163,20.578957,3354.37
6,2,Sci-Fi,163,20.493067,3340.37
7,1,Animation,161,20.387516,3282.39
8,1,Sci-Fi,149,21.795369,3247.51
9,1,Family,157,20.537771,3224.43


##### 13.We want to make sure you folks have a good handle on who your customers are. Please provide a list of all customer names, which store they go to, whether or not they are currently active, and their full addresses – street address, city, and country. 

In [31]:
pd.read_sql("""SELECT 
               customer.first_name, 
               customer.last_name, 
               customer.store_id,
               customer.active, 
               address.address, 
               city.city, 
               country.country
               FROM customer
               LEFT JOIN address ON customer.address_id = address.address_id
               LEFT JOIN city ON address.city_id = city.city_id
               LEFT JOIN country ON city.country_id = country.country_id;""",conn)

,first_name,last_name,store_id,active,address,city,country
0,MARY,SMITH,1,1,1913 Hanoi Way,Sasebo,Japan
1,PATRICIA,JOHNSON,1,1,1121 Loja Avenue,San Bernardino,United States
2,LINDA,WILLIAMS,1,1,692 Joliet Street,Athenai,Greece
3,BARBARA,JONES,2,1,1566 Inegl Manor,Myingyan,Myanmar
4,ELIZABETH,BROWN,1,1,53 Idfu Parkway,Nantou,Taiwan
...,...,...,...,...,...,...,...
594,TERRENCE,GUNDERSON,1,1,844 Bucuresti Place,Jinzhou,China
595,ENRIQUE,FORSYTHE,1,1,1101 Bucuresti Boulevard,Patras,Greece
596,FREDDIE,DUGGAN,1,1,1103 Quilmes Boulevard,Sullana,Peru
597,WADE,DELVALLE,1,1,1331 Usak Boulevard,Lausanne,Switzerland


##### 14.	We would like to understand how much your customers are spending with you, and also to know who your most valuable customers are. Please pull together a list of customer names, their total lifetime rentals, and the sum of all payments you have collected from them. It would be great to see this ordered on total lifetime value, with the most valuable customers at the top of the list. 

In [32]:
pd.read_sql(""" SELECT 
                customer.first_name, 
                customer.last_name, 
                COUNT(rental.rental_id) AS total_rentals, 
                SUM(payment.amount) AS total_payment_amount
                FROM customer
                LEFT JOIN rental ON customer.customer_id = rental.customer_id
                LEFT JOIN payment ON rental.rental_id = payment.rental_id
                GROUP BY 
                customer.first_name,
                customer.last_name
                ORDER BY 
                SUM(payment.amount) DESC;""", conn)

,first_name,last_name,total_rentals,total_payment_amount
0,KARL,SEAL,45,221.55
1,ELEANOR,HUNT,46,216.54
2,CLARA,SHAW,42,195.58
3,RHONDA,KENNEDY,39,194.61
4,MARION,SNYDER,39,194.61
...,...,...,...,...
594,ANNIE,RUSSELL,18,58.82
595,JOHNNY,TURPIN,19,57.81
596,BRIAN,WYMAN,12,52.88
597,LEONA,OBRIEN,14,50.86


##### 15. My partner and I would like to get to know your board of advisors and any current investors. Could you please provide a list of advisor and investor names in one table? Could you please note whether they are an investor or an advisor, and for the investors, it would be good to include which company they work with. 

In [33]:
pd.read_sql(""" SELECT
               'investor' AS type, 
                first_name, 
                last_name, 
                company_name
                FROM investor UNION 
                SELECT 
                'advisor' AS type, 
                first_name, 
                last_name, 
                NULL
                FROM advisor;""", conn)

,type,first_name,last_name,company_name
0,investor,Montgomery,Burns,Springfield Syndicators
1,investor,Anthony,Stark,Iron Investors
2,investor,William,Wonka,Chocolate Ventures
3,advisor,Barry,Beenthere,None
4,advisor,Cindy,Smartypants,None
5,advisor,Mary,Moneybags,None
6,advisor,Walter,White,None


##### 16. We're interested in how well you have covered the most-awarded actors. Of all the actors with three types of awards, for what % of them do we carry a film? And how about for actors with two types of awards? Same questions.  Finally, how about actors with just one award?

In [34]:
pd.read_sql("""SELECT
               CASE 
               WHEN actor_award.awards = 'Emmy, Oscar, Tony ' THEN '3 awards'
               WHEN actor_award.awards IN ('Emmy, Oscar','Emmy, Tony', 'Oscar, Tony') THEN '2 awards'
               ELSE '1 award'
               END AS number_of_awards, 
               AVG(CASE WHEN actor_award.actor_id IS NULL THEN 0 ELSE 1 END) AS pct_w_one_film
               FROM actor_award
               GROUP BY 
               CASE 
               WHEN actor_award.awards = 'Emmy, Oscar, Tony ' THEN '3 awards'
               WHEN actor_award.awards IN ('Emmy, Oscar','Emmy, Tony', 'Oscar, Tony') THEN '2 awards'
               ELSE '1 award'
               END """,conn)

,number_of_awards,pct_w_one_film
0,1 award,0.8333
1,2 awards,0.9242
2,3 awards,0.5714
